In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

# =====================================================
# 1️⃣ 데이터 로드 + 시간 파생
# =====================================================
df = pd.read_csv("new_flight_weather_merged.csv")

df["departure_datetime"] = pd.to_datetime(df["departure_datetime"])
df["dep_hour"] = df["departure_datetime"].dt.hour
df["dep_weekday"] = df["departure_datetime"].dt.weekday
df["is_weekend"] = df["dep_weekday"].isin([5, 6]).astype(int)

# =====================================================
# 2️⃣ 상태 → 다중분류 라벨 생성
# =====================================================
# 상태 컬럼 기준
# 정상운항 / 지연 / 회항 / 취소

label_map = {
    "정상운항": "문제없음",
    "지연": "지연",
    "회항": "회항",
    "취소": "취소"
}

df = df[df["상태"].isin(label_map.keys())].copy()
df["target"] = df["상태"].map(label_map)

print("클래스 분포 (원본)")
print(df["target"].value_counts())

# =====================================================
# 3️⃣ 컬럼 정의
# =====================================================
num_cols = ["기온(°C)", "풍속_ms", "dep_hour", "dep_weekday", "is_weekend"]
num_cols = [c for c in num_cols if c in df.columns]

cat_cols = ["공항명", "출발지", "도착지", "flight_type"]
cat_cols = [c for c in cat_cols if c in df.columns]

for c in cat_cols:
    df[c] = df[c].astype("category")

X_cols = num_cols + cat_cols

# =====================================================
# 4️⃣ Train / Test 분리 (시간 기준)
# =====================================================
df = df.sort_values("departure_datetime")
split_date = df["departure_datetime"].quantile(0.8)

train_df = df[df["departure_datetime"] <= split_date]
test_df  = df[df["departure_datetime"] > split_date]

print("Train:", len(train_df), "Test:", len(test_df))

# =====================================================
# 5️⃣ 🔥 다운사이징 (Train만)
#    - 문제없음 / 지연 ↓
#    - 회항 / 취소 그대로 유지
# =====================================================
train_ok   = train_df[train_df["target"] == "문제없음"]
train_delay = train_df[train_df["target"] == "지연"]
train_divert = train_df[train_df["target"] == "회항"]
train_cancel = train_df[train_df["target"] == "취소"]

# 기준 개수 = 지연 클래스
base_n = len(train_delay)

train_ok_down = train_ok.sample(n=base_n, random_state=42)
train_delay_down = train_delay.sample(n=base_n, random_state=42)

train_down = pd.concat([
    train_ok_down,
    train_delay_down,
    train_divert,
    train_cancel
]).sample(frac=1, random_state=42)

print("\n클래스 분포 (다운사이징 후)")
print(train_down["target"].value_counts())

# =====================================================
# 6️⃣ X / y 분리
# =====================================================
X_train = train_down[X_cols]
y_train = train_down["target"]

X_test  = test_df[X_cols]
y_test  = test_df["target"]

# =====================================================
# 7️⃣ LightGBM 다중분류 모델
# =====================================================
lgbm = LGBMClassifier(
    objective="multiclass",
    num_class=4,

    n_estimators=600,
    learning_rate=0.05,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,

    random_state=42,
    n_jobs=-1
)

lgbm.fit(
    X_train,
    y_train,
    categorical_feature=cat_cols
)

print("\n✅ LightGBM 다중분류 학습 완료")

# =====================================================
# 8️⃣ 평가
# =====================================================
y_pred = lgbm.predict(X_test)

print("\n📊 Multiclass Classification Report")
print(classification_report(y_test, y_pred))


C:\Users\Admin\AppData\Local\Temp\ipykernel_14512\427178239.py:9: DtypeWarning: Columns (29,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("new_flight_weather_merged.csv")


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\envs\4vector\lib\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_14512\427178239.py", line 9, in <module>
    df = pd.read_csv("new_flight_weather_merged.csv")
  File "C:\Users\Admin\anaconda3\envs\4vector\lib\site-packages\pandas\util\_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\Admin\anaconda3\envs\4vector\lib\site-packages\pandas\util\_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\Admin\anaconda3\envs\4vector\lib\site-packages\pandas\io\parsers\readers.py", line 950, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "C:\Users\Admin\anaconda3\envs\4vector\lib\site-packages\pandas\io\parsers\readers.py", line 611, in _read
    return parser.read(nrows)
  File "C:\Users\Admin\anaconda3\envs\4vecto

In [ ]:
# ===============================
# ✅ 한글 폰트 깨짐 방지 설정
# ===============================
import matplotlib.pyplot as plt
import platform
import matplotlib.font_manager as fm

system = platform.system()

if system == "Windows":
    # Windows (대부분 Malgun Gothic 사용)
    plt.rcParams["font.family"] = "Malgun Gothic"

elif system == "Darwin":
    # macOS
    plt.rcParams["font.family"] = "AppleGothic"

else:
    # Linux (Colab / 서버)
    # 나눔고딕이 없을 경우 대비
    try:
        plt.rcParams["font.family"] = "NanumGothic"
    except:
        plt.rcParams["font.family"] = "DejaVu Sans"

# 마이너스 기호 깨짐 방지
plt.rcParams["axes.unicode_minus"] = False

print(f"✅ 한글 폰트 설정 완료 ({system})")


In [ ]:
# =====================================================
# 9️⃣ (추가) 시각화용 표/그래프 모음
#    ✅ 기존 학습/평가 코드(위)는 "그대로" 두고,
#    ✅ 아래 셀만 실행하면 결과를 한눈에 볼 수 있도록 구성했습니다.
# =====================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix

# -----------------------------------------------------
# A) 필수 변수 체크 (위 셀에서 생성돼 있어야 합니다)
# -----------------------------------------------------
required_vars = ["lgbm", "X_test", "y_test", "y_pred"]
missing = [v for v in required_vars if v not in globals()]

if missing:
    print("⚠️ 아래 변수들이 현재 노트북 세션에 없습니다:", missing)
    print("👉 위의 학습/평가 셀을 먼저 실행한 뒤, 다시 이 셀을 실행하세요.")
else:
    # -------------------------------------------------
    # B) 클래스 순서(라벨) 결정
    #    - 혼동행렬/막대그래프에서 '항상 같은 순서'로 보기 위해 고정 순서 사용
    #    - 데이터에 없는 라벨은 자동 제거
    # -------------------------------------------------
    preferred_order = ["문제없음", "지연", "취소", "회항"]
    present_labels = list(pd.unique(pd.concat([
        pd.Series(y_test, name="y_test"),
        pd.Series(y_pred, name="y_pred")
    ])))

    labels = [c for c in preferred_order if c in present_labels]
    # 혹시 예상 외 라벨이 있을 경우를 대비해, 남은 라벨도 뒤에 붙임
    labels += [c for c in present_labels if c not in labels]

    print("✅ 사용 라벨 순서:", labels)

    # -------------------------------------------------
    # C) Classification Report → 표(DataFrame)로 보기
    #    - output_dict=True로 받아서 DataFrame으로 변환
    #    - zero_division=0: 분모 0인 경우(희귀 클래스) 0으로 처리
    # -------------------------------------------------
    report_dict = classification_report(
        y_test, y_pred,
        labels=labels,
        output_dict=True,
        zero_division=0
    )
    report_df = (
        pd.DataFrame(report_dict)
        .T
        .rename(columns={"f1-score": "f1"})
    )

    # 보기 좋게 소수점/정렬
    metric_cols = [c for c in ["precision", "recall", "f1", "support"] if c in report_df.columns]
    report_df = report_df[metric_cols].copy()
    report_df.loc[:, ["precision", "recall", "f1"]] = report_df[["precision", "recall", "f1"]].round(4)

    print("\n📌 Classification Report (표)")
    display(report_df)

    # -------------------------------------------------
    # D) 혼동행렬(Confusion Matrix)
    #    1) Raw(절대값)
    #    2) Normalized(행 기준 비율: 실제 클래스별로 어디로 흘렀는지)
    # -------------------------------------------------
    cm = confusion_matrix(y_test, y_pred, labels=labels)

    def plot_cm(matrix, title, normalize=False):
        """matplotlib만으로 혼동행렬을 그립니다(Seaborn 미사용)."""
        fig, ax = plt.subplots(figsize=(7, 6))
        im = ax.imshow(matrix, aspect="auto")

        ax.set_title(title)
        ax.set_xlabel("Predicted (예측)")
        ax.set_ylabel("True (실제)")

        ax.set_xticks(range(len(labels)))
        ax.set_yticks(range(len(labels)))
        ax.set_xticklabels(labels, rotation=45, ha="right")
        ax.set_yticklabels(labels)

        # 각 셀에 값 표시
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                val = matrix[i, j]
                if normalize:
                    txt = f"{val:.2f}"
                else:
                    txt = f"{int(val)}"
                ax.text(j, i, txt, ha="center", va="center")

        plt.colorbar(im, ax=ax)
        plt.tight_layout()
        plt.show()

    # 1) 절대값 혼동행렬
    plot_cm(cm, "Confusion Matrix (Raw Counts)", normalize=False)

    # 2) 정규화(행 기준)
    cm_row_sum = cm.sum(axis=1, keepdims=True)
    cm_norm = np.divide(cm, cm_row_sum, out=np.zeros_like(cm, dtype=float), where=cm_row_sum != 0)
    plot_cm(cm_norm, "Confusion Matrix (Row-normalized)", normalize=True)

    # -------------------------------------------------
    # E) 실제 분포 vs 예측 분포 (클래스 불균형/쏠림 확인)
    # -------------------------------------------------
    true_counts = pd.Series(y_test).value_counts().reindex(labels, fill_value=0)
    pred_counts = pd.Series(y_pred).value_counts().reindex(labels, fill_value=0)

    dist_df = pd.DataFrame({
        "true_count": true_counts,
        "pred_count": pred_counts
    })

    print("\n📌 실제/예측 분포 (표)")
    display(dist_df)

    # 막대그래프(같은 축에서 비교)
    fig, ax = plt.subplots(figsize=(8, 4))
    x = np.arange(len(labels))
    width = 0.35

    ax.bar(x - width/2, dist_df["true_count"], width, label="True")
    ax.bar(x + width/2, dist_df["pred_count"], width, label="Pred")

    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=0)
    ax.set_title("Class Distribution: True vs Pred")
    ax.set_ylabel("Count")
    ax.legend()
    plt.tight_layout()
    plt.show()

    # -------------------------------------------------
    # F) 클래스별 F1(막대그래프)
    #    - 어떤 클래스에서 성능이 무너지는지 빠르게 확인
    # -------------------------------------------------
    class_rows = [c for c in labels if c in report_df.index]
    f1_vals = report_df.loc[class_rows, "f1"].values

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(class_rows, f1_vals)
    ax.set_ylim(0, 1.0)
    ax.set_title("F1-score by Class")
    ax.set_ylabel("F1")
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

    # -------------------------------------------------
    # G) Feature Importance (LightGBM)
    #    - 모델이 어떤 피처를 중요하게 보는지 확인
    #    - 상위 TOP 20만 출력/시각화
    # -------------------------------------------------
    if hasattr(lgbm, "feature_importances_"):
        # feature 이름 확보 (우선순위: X_train.columns → X_cols → None)
        if "X_train" in globals() and hasattr(X_train, "columns"):
            feature_names = list(X_train.columns)
        elif "X_cols" in globals():
            feature_names = list(X_cols)
        else:
            feature_names = [f"f{i}" for i in range(len(lgbm.feature_importances_))]

        fi = pd.DataFrame({
            "feature": feature_names,
            "importance": lgbm.feature_importances_
        }).sort_values("importance", ascending=False)

        print("\n📌 Feature Importance TOP 20 (표)")
        display(fi.head(20))

        topk = fi.head(20).iloc[::-1]  # 그래프는 위→아래 읽기 편하게 역순
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.barh(topk["feature"], topk["importance"])
        ax.set_title("Feature Importance (Top 20)")
        ax.set_xlabel("Importance")
        plt.tight_layout()
        plt.show()
    else:
        print("\nℹ️ lgbm.feature_importances_ 를 찾을 수 없어 Feature Importance는 건너뜁니다.")

    # -------------------------------------------------
    # H) 예측 확률 기반 "자신감" 분포 (선택)
    #    - multiclass에서는 predict_proba로 각 클래스 확률을 얻을 수 있음
    #    - max_proba가 낮으면 모델이 헷갈리는 샘플이 많다는 뜻
    # -------------------------------------------------
    try:
        proba = lgbm.predict_proba(X_test)

        # proba가 (n_samples, n_classes) 형태라고 가정
        max_proba = np.max(proba, axis=1)

        fig, ax = plt.subplots(figsize=(8, 4))
        ax.hist(max_proba, bins=30)
        ax.set_title("Prediction Confidence (max class probability)")
        ax.set_xlabel("max_proba")
        ax.set_ylabel("Frequency")
        plt.tight_layout()
        plt.show()

        # 헷갈리는 샘플(자신감 낮은 순) TOP 10을 표로 확인
        # - 운영 관점에서 '사람이 검토할 후보'를 뽑을 때 유용
        hard_idx = np.argsort(max_proba)[:10]
        hard_df = pd.DataFrame({
            "index": hard_idx,
            "true": np.array(y_test)[hard_idx],
            "pred": np.array(y_pred)[hard_idx],
            "max_proba": max_proba[hard_idx]
        }).sort_values("max_proba")

        print("\n📌 모델이 가장 헷갈린 샘플 TOP 10 (표)")
        display(hard_df)

    except Exception as e:
        print("\nℹ️ predict_proba 기반 시각화는 현재 환경/모델 설정에서 실패했습니다.")
        print("에러:", repr(e))
